# ETL Example

In this Jupyter notebook, two ETL (Extract, Transform, Load) pipelines are performed, one which extracts data from a table on a web page (**Webscraping**) and one which extracts data from a server using an **API**.

Before running this notebook, please ensure the Python packages being loaded below are installed on your system.

In [1]:
# Load the required libraries for this task

from bs4 import BeautifulSoup
import html5lib
import requests
import pandas as pd
import json

## Example 1: Webscraping

We will use the Wikipedia web page listing the largest banks in the world, based on total asset value.

### Extract

In [2]:
html_data = requests.get("https://en.wikipedia.org/wiki/List_of_largest_banks").text

# html_data

### Transform

In [3]:
soup = BeautifulSoup(html_data, 'html5lib')

soup = soup.find(id="By_total_assets")
soup = soup.parent
soup = soup.next_sibling
soup = soup.next_sibling
soup = soup.next_sibling
soup = soup.next_sibling
soup = soup.next_sibling

# soup

In [4]:
largest_banks_df = pd.DataFrame(columns=["Rank", "Bank name", "Total assets as of 2021 (US$ billion)"])

for row in soup.tbody.find_all("tr"):
    col = row.find_all("td")
    if (col != []):
        rank = col[0].text.strip()
        bank_name = col[1].text.strip()
        total_assets = col[2].text.strip()
        largest_banks_df = largest_banks_df.append({
            "Rank": rank,
            "Bank name": bank_name,
            "Total assets as of 2021 (US$ billion)": total_assets
        }, ignore_index=True)

largest_banks_df

,Rank,Bank name,Total assets as of 2021 (US$ billion)
0,1,Industrial and Commercial Bank of China Limited,"5,536.53"
1,2,China Construction Bank,"4,762.46"
2,3,Agricultural Bank of China,"4,575.95"
3,4,Bank of China,"4,206.53"
4,5,JPMorgan Chase,"3,743.57"
...,...,...,...
95,96,Raiffeisen Gruppe,312.07
96,97,Swedbank,304.20
97,98,Bayerische Landesbank,303.14
98,99,Banco Bradesco,300.73


### Load

In [5]:
largest_banks_df.to_csv("largest_banks.csv", index=False)
largest_banks_df.to_json("largest_banks.json")

# Example 2: API

This example uses the APILayer Exchange Rates Data REST API to save ***current*** GBP exchange rate data to a Python object directly from a server.

### Extract

In [6]:
# Write your code here
# API key: H3WJWPpeIOyQxjOQdrEczX2RfmMAluTX

url = "https://api.apilayer.com/exchangerates_data/latest?base=GBP&apikey=H3WJWPpeIOyQxjOQdrEczX2RfmMAluTX" #Make sure to change ******* to your API key.

url_text = requests.get(url).text

# url_text

### Transform

In [7]:
exchange_rate_dict = json.loads(url_text)['rates']

# exchange_rate_dict

exchange_rate_currencies = []
exchange_rate_rates = []

for currency in exchange_rate_dict.keys():
    exchange_rate_currencies.append(currency)

for rate in exchange_rate_dict.values():
    exchange_rate_rates.append(rate)

exchange_rate_dict2 = {"Currency": [], "Rate": []}
exchange_rate_dict2["Currency"] = exchange_rate_currencies
exchange_rate_dict2["Rate"] = exchange_rate_rates

exchange_rate_df = pd.DataFrame(exchange_rate_dict2)

exchange_rate_df2 = exchange_rate_df
exchange_rate_df2 = exchange_rate_df2.set_index("Currency")

exchange_rate_df2

,Rate
Currency,
AED,4.465246
AFN,107.593267
ALL,129.934442
AMD,472.807093
ANG,2.193006
...,...
YER,304.335424
ZAR,22.113844
ZMK,10943.310097


### Load

In [8]:
exchange_rate_df2.to_csv("exchange_rate.csv")
exchange_rate_df2.to_json("exchange_rate.json")